In [1]:
!pip install groq python-dotenv numpy tqdm datasets


[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from groq import Groq
from dotenv import load_dotenv
from datasets import load_dataset

import os
from tqdm import tqdm
import re
import random
import pprint

from typing import List, Dict, Any

load_dotenv()
random.seed(0)

client = Groq()
gsm8k_dataset = load_dataset("gsm8k", "main")

gsm8k_train = gsm8k_dataset["train"]
gsm8k_test  = gsm8k_dataset["test"]

/Users/yezzi/Desktop/[7회차] LLM 과제/ybigtaLLM/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def generate_response_using_Llama(
        prompt: str,
        model: str = "llama3-8b-8192"
    ):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that solves math problems."
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            model=model,
            temperature=0.3, ### 수정해도 됩니다!
            stream=False
        )
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        print(f"API call error: {str(e)}")
        return None

#### 응답 잘 나오는지 확인해보기

In [4]:
response = generate_response_using_Llama(
    prompt="Hello world!",
)
print(response)

Hello! I'm excited to help you with any math problems you have. What kind of math are you working on? Do you have a specific problem you're stuck on or a concept you're trying to understand? Let me know and I'll do my best to assist you!


#### GSM8K 데이터셋 확인해보기

In [5]:
print("[Question]")
for l in gsm8k_test['question'][0].split("."):
    print(l)
print("="*100)
print("[Answer]")
print(gsm8k_test['answer'][0])

[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


#### Util 함수들
- extract_final_answer: LLM의 응답을 parse하여 최종 결과만 추출 (정답과 비교하기 위해)
- run_benchmark_test: 벤치마크 테스트
- save_final_result: 결과물 제출을 위한 함수

In [7]:
### 수정해도 됩니다!
def extract_final_answer(response: str):
    regex = r"(?:Answer:|Model response:)\s*\$?([0-9,]+)\b|([0-9,]+)\s*(meters|cups|miles|minutes)"
    matches = re.finditer(regex, response, re.MULTILINE)
    results = [match.group(1) if match.group(1) else match.group(2).replace(",", "") for match in matches]

    if len(results) == 0:
        additional_regex = r"\$?([0-9,]+)"
        additional_matches = re.finditer(additional_regex, response, re.MULTILINE)
        results.extend([match.group(1).replace(",", "") for match in additional_matches])

    return results[-1] if results else None

In [9]:
import re
from tqdm import tqdm

def run_benchmark_test(
    dataset,
    prompt: str,
    model: str = "llama3-8b-8192",
    num_samples: int = 50,
    VERBOSE: bool = False
):
    correct = 0
    total = 0
    results = []

    for i in tqdm(range(min(num_samples, len(dataset)))):
        question = dataset[i]["question"]
        correct_answer = float(re.findall(r'\d+(?:\.\d+)?', dataset[i]["answer"].split('####')[-1])[0])

        response = generate_response_using_Llama(
            prompt=prompt.format(question=question),
            model=model
        )

        predicted_answer = None
        is_correct = False

        if response:
            if VERBOSE:
                print("=" * 50)
                print(response)
                print("=" * 50)

            predicted_answer = extract_final_answer(response)

            if predicted_answer is not None and predicted_answer.strip() != "":
                try:
                    predicted_answer = float(predicted_answer.replace(",", ""))
                    diff = abs(predicted_answer - correct_answer)
                    is_correct = diff < 1e-5
                except ValueError:
                    predicted_answer = None
                    is_correct = False

        if is_correct:
            correct += 1

        total += 1
        results.append({
            'question': question,
            'correct_answer': correct_answer,
            'predicted_answer': predicted_answer,
            'response': response,
            'correct': is_correct
        })

        if (i + 1) % 5 == 0:
            current_acc = correct / total if total > 0 else 0
            print(f"Progress: [{i+1}/{num_samples}]")
            print(f"Current Acc.: [{current_acc:.2%}]")

    return results, correct / total if total > 0 else 0


In [10]:
def save_final_result(results: List[Dict[str, Any]], accuracy: float, filename: str) -> None:
    result_str = f"====== ACCURACY: {accuracy} ======\n\n"
    result_str += f"[Details]\n"
    
    for idx, result in enumerate(results):
        result_str += f"Question {idx+1}: {result['question']}\n"
        result_str += f"Correct Answer: {result['correct_answer']}\n"
        result_str += f"Predicted Answer: {result['predicted_answer']}\n"
        result_str += f"Correct: {result['correct']}\n\n"
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(result_str)

#### Direct prompting with few-shot example

In [11]:
def construct_direct_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )

    prompt = "Instruction:\nSolve the following mathematical question and generate ONLY the answer after a tag, 'Answer:' without any rationale.\n"

    for i in range(num_examples):
        cur_question = train_dataset['question'][i]
        cur_answer = train_dataset['answer'][i].split("####")[-1].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [12]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_direct_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=10
)
save_final_result(results, accuracy, "example.txt")

 50%|█████     | 5/10 [00:03<00:03,  1.37it/s]

Progress: [5/10]
Current Acc.: [20.00%]


100%|██████████| 10/10 [00:06<00:00,  1.57it/s]

Progress: [10/10]
Current Acc.: [20.00%]


In [13]:
# TODO: 0 shot, 3 shot, 5 shot direct prompting을 통해 벤치마크 테스트를 한 후, 각각 direct_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 direct_prompting_5.txt
# 항상 num_samples=50 입니다!
for shot in [0, 3, 5]:
    print(f"▶ Running Direct Prompting with {shot}-shot examples")
    
    prompt = construct_direct_prompt(shot)
    
    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=prompt,
        num_samples=50,  # 반드시 50
        VERBOSE=False
    )
    
    filename = f"direct_prompting_{shot}.txt"
    save_final_result(results, accuracy, filename)

    print(f"Saved: {filename} / Accuracy: {accuracy:.2%}\n")


▶ Running Direct Prompting with 0-shot examples


 10%|█         | 5/50 [00:04<00:31,  1.42it/s]

Progress: [5/50]
Current Acc.: [20.00%]


 20%|██        | 10/50 [00:06<00:23,  1.68it/s]

Progress: [10/50]
Current Acc.: [20.00%]


...

Progress: [40/50]
Current Acc.: [20.00%]


 90%|█████████ | 45/50 [01:59<00:12,  2.49s/it]

Progress: [45/50]
Current Acc.: [17.78%]


100%|██████████| 50/50 [02:15<00:00,  2.70s/it]

Progress: [50/50]
Current Acc.: [20.00%]
Saved: direct_prompting_5.txt / Accuracy: 20.00%



### Chain-of-Thought prompting with few-shot example
```text
[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
====================================================================================================
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18
```

[Answer] 아래의 정답을 도출하는 과정을 예시로 달아주면 CoT의 few shot이 되겠죠?

In [14]:
def construct_CoT_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )
    prompt = "CoT PROMPT" #TODO: 프롬프트를 작성해주세요!
    prompt += "Instruction:\nSolve the following math problems step-by-step. Show your reasoning clearly before giving the final answer.\n"
    
    for i, idx in enumerate(sampled_indices):
        question = train_dataset['question'][idx]
        answer = train_dataset['answer'][idx]

        reasoning = answer.split("####")[0].strip()
        final_answer = answer.split("####")[-1].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{question}\n"
        prompt += f"Answer:\nLet's think step by step.\n{reasoning}\nThe answer is {final_answer}.\n"


    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [15]:
# TODO: 0 shot, 3 shot, 5 shot CoT prompting을 통해 벤치마크 테스트를 한 후, 각각 CoT_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 CoT_prompting_5.txt
# 항상 num_samples=50 입니다!
for shot in [0, 3, 5]:
    print(f"▶ Running CoT Prompting with {shot}-shot examples")
    
    prompt = construct_CoT_prompt(num_examples=shot)

    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=prompt,
        num_samples=50,
        VERBOSE=False
    )

    filename = f"CoT_prompting_{shot}.txt"
    save_final_result(results, accuracy, filename)

    print(f"Saved: {filename} / Accuracy: {accuracy:.2%}\n")


▶ Running CoT Prompting with 0-shot examples


 10%|█         | 5/50 [00:07<01:34,  2.09s/it]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [00:15<01:08,  1.71s/it]

Progress: [10/50]
Current Acc.: [60.00%]


...

Progress: [40/50]
Current Acc.: [60.00%]


 90%|█████████ | 45/50 [09:50<01:21, 16.35s/it]

Progress: [45/50]
Current Acc.: [60.00%]


100%|██████████| 50/50 [10:37<00:00, 12.75s/it]

Progress: [50/50]
Current Acc.: [62.00%]
Saved: CoT_prompting_5.txt / Accuracy: 62.00%



### Construct your prompt!!

목표: 본인만의 프롬프트를 통해 정답률을 더 끌어올려보기!
- gsm8k의 train 데이터셋에서 예시를 가져온 다음 (자유롭게!)
- 그 예시들에 대한 풀이 과정을 만들어주세요!
- 모든 것들이 자유입니다! Direct Prompting, CoT Prompting을 한 결과보다 정답률만 높으면 돼요.

In [26]:
def construct_my_prompt(example_list: List[Dict[str, str]], num_examples: int = 3) -> str:
    import re

    prompt = (
        "You are a highly capable math solver.\n"
        "When given a math word problem, solve it clearly and precisely using step-by-step reasoning.\n"
        "Do NOT write any introductions like 'Sure!' or 'Let's solve it!'.\n"
        "Start reasoning immediately.\n"
        "At the end, always write the final answer **on a new line**, in this exact format:\n"
        "Answer: <number>\n"
        "The answer must be a number only, with no extra explanation.\n"
        "Let's think step by step.\n\n"
    )

    for i, example in enumerate(example_list[:num_examples]):
        question = example["question"].strip()
        answer = example["answer"].strip()

        # Extract final numeric answer (e.g., from "#### 24")
        match = re.search(r"####\s*(.*)", answer)
        final = match.group(1).strip() if match else "UNKNOWN"

        reasoning = answer.split("####")[0].strip()

        prompt += f"[Example {i+1}]\n"
        prompt += f"Q: {question}\n"
        prompt += f"A: {reasoning}\n"
        prompt += f"Answer: {final}\n\n"

    prompt += "---\nQ: {question}\nA:"
    return prompt


In [ ]:
# TODO: 만든 0 shot, 3 shot, 5 shot example과 프롬프트를 통해 벤치마크 테스트를 한 후, 각각 My_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 My_prompting_5.txt
# 항상 num_samples=50 입니다!
for shot in [0, 3, 5]:
    print(f"▶ Running My Prompting with {shot}-shot examples")

    # train 데이터 중 50개 샘플에서 shot만큼 추출
    example_list = list(gsm8k_train.select(range(50)))

    prompt = construct_my_prompt(example_list, num_examples=shot)

    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=prompt,
        num_samples=50,
        VERBOSE=False
    )

    filename = f"My_prompting_{shot}.txt"
    save_final_result(results, accuracy, filename)

    print(f"Saved: {filename} / Accuracy: {accuracy:.2%}\n")


### 보고서 작성하기
#### 아래의 내용이 포함되면 됩니다!

1. Direct Prompting, CoT Prompting, My Prompting을 0 shot, 3 shot, 5 shot 정답률을 표로 보여주세요!
2. CoT Prompting이 Direct Prompting에 비해 왜 좋을 수 있는지에 대해서 서술해주세요!
3. 본인이 작성한 프롬프트 기법이 CoT에 비해서 왜 더 좋을 수 있는지에 대해서 설명해주세요!
4. 최종적으로, `PROMPTING.md`에 보고서를 작성해주세요!